In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, preprocessing, metrics
df_train = pd.read_csv('../Data_with_NumMosquitos/unique_train.csv')
#df_train_nomosq = pd.read_csv('../train_filled_new.csv')
df_test = pd.read_csv('../Data_with_NumMosquitos/new_test_priors.csv')
#df_test_nomosq = pd.read_csv('../test_filled_new.csv')

df_train = pd.read_csv('../Data_with_NumMosquitos/train_plus_pred.csv')
#df_train_nomosq = pd.read_csv('../train_filled_new.csv')
df_test = pd.read_csv('../Data_with_NumMosquitos/test_plus_pred.csv')
#df_test_nomosq = pd.read_csv('../test_filled_new.csv')

In [2]:
random_state = 42
df_train = df_train.drop(['Block'],axis=1)
df_test = df_test.drop(['Block'],axis=1)

ValueError: labels ['Block'] not contained in axis

In [3]:
from sklearn.cross_validation import train_test_split, ShuffleSplit,cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc

def compute_score(yTrue,preds):
    
    #return roc_auc_score(yTrue,preds)
    fpr,tpr,thresholds = roc_curve(yTrue,preds)
    return auc(fpr,tpr)

def CrossValidate(clf,X,y):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.9,random_state=random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict_proba(X_test)[:,1]
    print(compute_score(y_test,yhat))
    
    


In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
#clf = RandomForestClassifier(n_estimators = 1000,n_jobs = 2,max_features=32, verbose=2,random_state=random_state)

clf = RandomForestClassifier(n_estimators=1200,random_state = 42,
                            max_features = 22,
                            criterion = 'entropy',
                            #warm_start = True,
                            min_samples_split = 1,
                            #max_leaf_nodes = 10,
                            max_depth = 15,
                            n_jobs=3
                            )

clf = RandomForestClassifier(n_estimators=1022,random_state = 42,
                            max_features = 22,
                            criterion = 'entropy',
                            #warm_start = True,
                            min_samples_split = 1,
                            #max_leaf_nodes = 10,
                            max_depth = 15,
                            n_jobs=3
                            )

#clf = AdaBoostClassifier(n_estimators=2000,learning_rate = .7,random_state=random_state)

In [6]:
df_train.dtypes[df_train.dtypes == 'object']

Series([], dtype: object)

In [7]:
df_test.dtypes[df_test.dtypes == 'object']

Series([], dtype: object)

In [8]:
df_test = df_test.drop(['Unnamed: 0'],axis=1)

In [9]:
#strlist = ['Date','Unnamed: 0','Unnamed: 0.1','Id']

#df_test = df_test.drop(strlist,axis=1)
#df_train = df_train.drop(strlist,axis=1)

In [10]:
df_train.Species.values, df_test.Species.values

(array([2, 3, 3, ..., 2, 2, 2]), array([2, 3, 1, ..., 5, 2, 0]))

In [11]:
#species_encoder = preprocessing.LabelEncoder()

#df_train.Species = species_encoder.fit_transform(df_train.Species)
from sklearn.preprocessing import scale
X = scale(df_train)
X_test = scale(df_test)

In [12]:
[var for var in df_train if var not in df_test]

['WnvPresent_DateTrapSpecies']

In [13]:
[var for var in df_test if var not in df_train]

[]

In [14]:
y = df_train.WnvPresent_DateTrapSpecies

#string_features = ['Date','Address','Street','AddressNumberAndStreet','CodeSum']
#df_train = df_train.drop(string_features,axis= 1)
#df_test = df_test.drop(string_features,axis=1)
#df_test = df_test.drop(st_f[1:],axis=1)
df_train2 = df_train.drop('count_mosquitos',axis=1)
df_test2 = df_test.drop('count_mosquitos',axis=1)
X_list = [(df_train.drop('WnvPresent_DateTrapSpecies',axis=1),df_test),
          (df_train2.drop('WnvPresent_DateTrapSpecies',axis=1), df_test2)]


In [15]:
df_train.WnvPresent_DateTrapSpecies

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
8491    0
8492    0
8493    0
8494    0
8495    0
8496    0
8497    0
8498    0
8499    1
8500    0
8501    1
8502    0
8503    1
8504    1
8505    0
8506    0
8507    0
8508    0
8509    0
8510    0
8511    0
8512    0
8513    0
8514    0
8515    0
8516    1
8517    0
8518    0
8519    0
8520    0
Name: WnvPresent_DateTrapSpecies, dtype: int64

In [16]:
CrossValidate(clf,df_train.drop('WnvPresent_DateTrapSpecies',axis=1),y)

0.852740087318


In [17]:
np.mean(y)

0.085905410163126389

In [114]:
df_support = pd.read_csv('../test.csv')
df_support.Species.values

array(['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS', ...,
       'CULEX TARSALIS', 'UNSPECIFIED CULEX', 'CULEX ERRATICUS'], dtype=object)

In [115]:
import numpy as np
i = 1
for X,X_test in X_list:
    i +=  1
    clf.fit(X,y)
    
    testtest = clf.predict_proba(X_test)
    yhat = testtest[:,1]
    yhat = pd.DataFrame(yhat,columns=['WnvPresent'])
    yhat[df_support.Species=='CULEX TARSALIS']=0
    yhat[df_support.Species=='CULEX ERRATICUS']=0
    yhat[df_support.Species=='UNSPECIFIED CULEX']=0

    yhat.index +=1

    #yhat[df_support.Species=='CULEX TARSALIS']=0
    print(np.mean(yhat))
    yhat.to_csv(str(i) + 'submission.csv',index_label = 'Id')

WnvPresent    0.099161
dtype: float64
WnvPresent    0.093531
dtype: float64


In [116]:
yhat.mean()

WnvPresent    0.093531
dtype: float64

In [117]:
yhat.head()

,WnvPresent
1,0.005871
2,0.003914
3,0.005871
4,0.003914
5,0.003914


In [118]:
yhat

,WnvPresent
1,0.005871
2,0.003914
3,0.005871
4,0.003914
5,0.003914
6,0.000000
7,0.000000
8,0.000000
9,0.003914
10,0.002935
